In [1]:
import xarray as xr
import backgroung as bg
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import cartopy.crs as ccrs
import bgcalc_xr as bgcalc
import cmocean as cm

/Users/vrx/miniconda3/envs/owl/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
model_path = 'http://copano.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg'
ds = xr.open_dataset(model_path)

In [6]:
box_xl_rho = xr.zeros_like(ds.h, dtype=bool)
box_xl_rho[70:140,264:376] = 1

box_l_rho = xr.zeros_like(ds.h, dtype=bool)
box_l_rho[71:141, 290:346] = 1
box_l_psi = xr.zeros_like(ds.h, dtype=bool)

box_m_rho = xr.zeros_like(ds.h, dtype=bool)
box_m_rho[79:133, 300:337] = 1

box_s_rho = xr.zeros_like(ds.h, dtype=bool)
box_s_rho[98:114, 313:326] = 1

box_xs_rho = xr.zeros_like(ds.h, dtype=bool)
box_xs_rho[104:109, 318:322] = 1

box_2c_rho = xr.zeros_like(ds.h, dtype=bool)
box_2c_rho[106:108, 320] = 1

dA = 1/(ds.pm*ds.pn)
area_bxl = dA.where(box_xl_rho).sum()
area_bl = dA.where(box_l_rho).sum()
area_bm = dA.where(box_m_rho).sum()
area_bs = dA.where(box_s_rho).sum()
area_bxs = dA.where(box_xs_rho).sum()
area_2c = dA.where(box_2c_rho).sum()

print(area_bxl/1e6, (1./ds.pm.where(box_xl_rho)).sum(dim='xi_rho').mean())
print(area_bl/1e6, (1./ds.pm.where(box_l_rho)).sum(dim='xi_rho').mean())
print(area_bm/1e6, (1./ds.pm.where(box_m_rho)).sum(dim='xi_rho').mean())
print(area_bs/1e6, (1./ds.pm.where(box_s_rho)).sum(dim='xi_rho').mean())
print(area_bxs/1e6, (1./ds.pm.where(box_xs_rho)).sum(dim='xi_rho').mean())
print(area_2c/1e6, (1./ds.pm.where(box_2c_rho)).sum(dim='xi_rho').mean())

<xarray.DataArray ()>
array(19526.99303898834) <xarray.DataArray ()>
array(79441.383767252)
<xarray.DataArray ()>
array(9962.005625938564) <xarray.DataArray ()>
array(40147.187164069655)
<xarray.DataArray ()>
array(5032.272047239942) <xarray.DataArray ()>
array(20450.351628127126)
<xarray.DataArray ()>
array(516.2210762980782) <xarray.DataArray ()>
array(2124.778436440571)
<xarray.DataArray ()>
array(49.53977221795978) <xarray.DataArray ()>
array(204.36487221429854)
<xarray.DataArray ()>
array(4.948902571702891) <xarray.DataArray ()>
array(20.44208017896111)


In [16]:
xr.where((1./ds.pm).where(box_xl_rho).sum(dim='xi_rho')>0.)

TypeError: where() missing 2 required positional arguments: 'x' and 'y'

In [ ]:
hmin = 12.
hmax = 50.
mab = 10.

lonl = -95
lonr = -91
latl = 28.
latu = 29.6

mask =(ds.h>=hmin)&(ds.h<=hmax)& \
    (ds.coords['lon_rho']>=lonl)&(ds.coords['lon_rho']<=lonr)& \
    (ds.coords['lat_rho']>=latl)& (ds['lat_rho'] < latu)& \
    (ds.mask_rho==1)
mask[161:166,274:279]=False
ds.coords['mask2D'] = (('eta_rho', 'xi_rho'), mask)
ds = ds.sel(ocean_time=slice("2010-08-01T00:00:00", "2010-09-30T23:30:00"))
budget = bgcalc_xr.Calculator(ds)


In [ ]:
tidx = 0
budget.subset(tidx)

In [ ]:
i_mx = budget.integ_matrix()
rate = budget.integrate(budget.get_rate(), i_mx = i_mx)
SOD = budget.get_SOD()
uOxadv,vOxadv = budget.get_inthadv()
hadv = budget.integrate(uOxadv + vOxadv, i_mx=i_mx)
vadv = budget.get_advflux_z()
vdiff = budget.get_difflux_z(btm_diff=SOD)

In [ ]:
new_ds = xr.Dataset({'rate': rate, 'hadv': hadv, 'vadv': vadv, 'vdiff': vdiff, 'SOD': SOD})
new_ds.to_netcdf('calc_bg_'+ str(new_ds.ocean_time.data)[:13] + '.nc')